In [13]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os


In [14]:


# List all CSV files in the current directory
csv_files = [os.path.join('./GestureDatasetBigger', f) for f in os.listdir('./GestureDatasetBigger') if f.endswith('_gestures.csv')]


# Load and combine
all_data = pd.concat(
    [pd.read_csv(file) for file in csv_files],
    ignore_index=True
)

# Check the structure
print(all_data.head())

                              gestureId username   shape  pointIndex   x  \
0  1c32e835-15ca-4fd2-ada0-654541470796    James  square           0  66   
1  1c32e835-15ca-4fd2-ada0-654541470796    James  square           1  67   
2  1c32e835-15ca-4fd2-ada0-654541470796    James  square           2  68   
3  1c32e835-15ca-4fd2-ada0-654541470796    James  square           3  71   
4  1c32e835-15ca-4fd2-ada0-654541470796    James  square           4  75   

           y          time  velocityX  velocityY    speed  gestureStartTime  \
0  76.671875  1.748716e+09      0.000      0.000    0.000      1.748716e+09   
1  77.671875  1.748716e+09     55.556     55.556   78.568      1.748716e+09   
2  77.671875  1.748716e+09    124.998      0.000  124.998      1.748716e+09   
3  77.671875  1.748716e+09    333.340      0.000  333.340      1.748716e+09   
4  78.671875  1.748716e+09    799.982    199.995  824.602      1.748716e+09   

   gestureEndTime  gestureDuration  
0    1.748716e+09            1.

In [15]:
import numpy as np
from scipy.interpolate import interp1d

def preprocess_gestures_with_velocity(df, target_num_points=16, target_width=50, target_height=40):
    processed_gestures = []
    gesture_labels = []
    gesture_users = []

    for gesture_id, group in df.groupby('gestureId'):
        x = group['x'].values
        y = group['y'].values
        velocity_x = group['velocityX'].values
        velocity_y = group['velocityY'].values

        # Center at geometric center
        center_x = np.mean(x)
        center_y = np.mean(y)
        x_centered = x - center_x
        y_centered = y - center_y

        # Scale proportionally
        max_x = np.max(np.abs(x_centered))
        max_y = np.max(np.abs(y_centered))
        scale_x = target_width / (2 * max_x) if max_x > 0 else 1.0
        scale_y = target_height / (2 * max_y) if max_y > 0 else 1.0
        scale_factor = min(scale_x, scale_y)
        x_scaled = x_centered * scale_factor
        y_scaled = y_centered * scale_factor

        # Calculate cumulative distances
        distances = np.sqrt(np.diff(x_scaled)**2 + np.diff(y_scaled)**2)
        cumulative_distance = np.insert(np.cumsum(distances), 0, 0)
        total_distance = cumulative_distance[-1]
        if total_distance == 0:
            continue

        # Remove duplicate cumulative distances to avoid interp1d issues
        _, unique_indices = np.unique(cumulative_distance, return_index=True)
        cumulative_distance = cumulative_distance[unique_indices]
        x_scaled = x_scaled[unique_indices]
        y_scaled = y_scaled[unique_indices]
        velocity_x = velocity_x[unique_indices]
        velocity_y = velocity_y[unique_indices]

        if len(cumulative_distance) < 2:
            continue  # Not enough points to interpolate

        # Interpolate to target_num_points
        target_distances = np.linspace(0, total_distance, target_num_points)
        interp_func_x = interp1d(cumulative_distance, x_scaled, kind='linear', fill_value="extrapolate")
        interp_func_y = interp1d(cumulative_distance, y_scaled, kind='linear', fill_value="extrapolate")
        interp_func_vx = interp1d(cumulative_distance, velocity_x, kind='linear', fill_value="extrapolate")
        interp_func_vy = interp1d(cumulative_distance, velocity_y, kind='linear', fill_value="extrapolate")
        x_resampled = interp_func_x(target_distances)
        y_resampled = interp_func_y(target_distances)
        vx_resampled = interp_func_vx(target_distances)
        vy_resampled = interp_func_vy(target_distances)

        # Combine features: [x1, y1, vx1, vy1, ..., xN, yN, vxN, vyN]
        gesture_features = np.column_stack((x_resampled, y_resampled, vx_resampled, vy_resampled))
        processed_gestures.append(gesture_features)
        gesture_labels.append(group['shape'].iloc[0])
        gesture_users.append(group['username'].iloc[0])

    return np.array(processed_gestures), gesture_labels, gesture_users


In [16]:
processed_gestures, gesture_labels, gesture_users = preprocess_gestures_with_velocity(all_data)
X = processed_gestures.reshape((processed_gestures.shape[0], -1))  # Flatten


In [17]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(gesture_labels)


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [19]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [20]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Test Accuracy: 0.97
Classification Report:
              precision    recall  f1-score   support

      circle       0.94      0.98      0.96        50
      square       0.98      0.98      0.98        50
    triangle       0.98      0.94      0.96        50

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150

Confusion Matrix:
[[49  1  0]
 [ 0 49  1]
 [ 3  0 47]]


In [21]:
import joblib

# Save the kNN model
joblib.dump(rf, 'random_forest_model.pkl')

# Save the label encoder
# joblib.dump(label_encoder, 'label_encoder.pkl')

['random_forest_model.pkl']